In [1]:
import json
import os
import re
import pandas as pd
import numpy as np
from sys import argv
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import google.generativeai as genai 
import google.ai.generativelanguage as glm
from sentence_transformers import SentenceTransformer
import typing_extensions as typing
import enum

d:\Anaconda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('./secrets.json') as f:
    secrets = json.loads(f.read())
GOOGLE_API_KEY = secrets['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)

safe = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [3]:
emb_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
model = genai.GenerativeModel('gemini-1.5-flash-latest', safety_settings=safe)

d:\Anaconda\envs\llm\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# 특정 요일 방문 횟수 비중을 나타내는 TypeDict
class VisitCountShare(typing.TypedDict):
    day: str          # 요일 (예: '월요일')
    visit_percentage: float  # 해당 요일의 방문 비중
    

class Category(enum.Enum):
    NONE = "None"
    Homestyle = "가정식"
    Chinese = "중식"
    SingleMenu = "단품요리 전문점"
    Coffee = "커피"
    
# Enum은 이름이 아니라 내용을 보고 구분하는듯. 아마 직접 소비건수 상위 ... 이런식으로 하면 알아먹긴할듯
class Range(enum.Enum):
    NONE = "None"
    Top10Percent = "상위 10% 미만"
    Top10To25Percent = "상위 10%이상 25%미만"
    Top25To50Percent = "상위 25%이상 50%미만"
    Top50To75Percent = "상위 50%이상 75%미만"
    Top75To90Percent = "상위 75%이상 90%미만"
    Top90PercentOver = "상위 90% 초과"
    

class orderType(enum.Enum):
    NONE = "None"
    highest = "highest"
    lowest = "lowest"
    
class filterType(enum.Enum):
    NONE = "None"
    Mon = "월요일이용비중"
    Tue = "화요일이용비중"
    Wed = "수요일이용비중"
    Thu = "목요일이용비중"
    Fri = "금요일이용비중"
    Sat = "토요일이용비중"
    Sun = "일요일이용비중"
    HR_5_11 = "5시11시이용건수비중"
    HR_12_13 = "12시13시이용건수비중"
    HR_14_17 = "14시17시이용건수비중"
    HR_18_22 = "18시22시이용건수비중"
    HR_23_4 = "23시4시이용건수비중"
    Local = "현지인이용건수비중"
    Mal = "남성회원수비중"
    Fme = "여성회원수비중"
    Age_20_Und = "20대이하회원수비중"
    Age_30 = "30대회원수비중"
    Age_40 = "40대회원수비중"
    Age_50 = "50대회원수비중"
    Age_60_Ovr = "60대이상회원수비중"
    
# 필터와 정렬 정보를 위한 TypedDict
class FilterOrder(typing.TypedDict):
    filter_type: filterType  # 필터 종류 (예: 요일, 성별 등)
    order_type: orderType   # 정렬 타입 (예: 'highest', 'lowest')

class Query(typing.TypedDict):
    address: str
    category: Category
    Usage_Count_Range: str
    Spending_Amount_Range: str
    Average_Spending_Amount_Range: str # Range쓰면 분류 잘못함.
    # Visit_count_specific: VisitCountShare
    # Local_Visitor_Proportion: float
    ranking_condition: FilterOrder
    
    
    
    
    

In [5]:
print(list(Query.__annotations__.keys())[:-1])

['address', 'category', 'Usage_Count_Range', 'Spending_Amount_Range', 'Average_Spending_Amount_Range']


In [6]:
chat = model.start_chat(history=[])
question = "제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 15%에 속하고 건당평균이용금액구간이 상위 20%에 속하면서 소비구간이 상위 80%이고 토요일 이용비중이 제일 낮은곳은?"
# question = "제주시 상모리에 짜장면집 추천해주라"
prompt = f"다음 질문에서 요구사항을 보고 모든 항목({list(Query.__annotations__.keys())[:-1]})을 반드시 출력해라. ranking_condition는 없을 수도 있으며, 오직 순위를 나타내는 조건(가장 큰것, 가장 작은것)에만 해당한다. 주소는 질문에서 찾는다. 모르는 내용을 넣지 마라.\n질문: {question}"

response = chat.send_message(prompt, generation_config=genai.GenerationConfig(
        response_mime_type="application/json", response_schema=list[Query]
    ),)
# print(response)
output = response.parts[0].text
print(output)
json_datas = json.loads(response.parts[0].text)
print("객체 개수", len(json_datas))
if len(json_datas) > 1:
    print(json_datas[0])
    print(json_datas[1])

[{"address": "제주시 노형동", "category": "단품요리 전문점", "ranking_condition": {"filter_type": "토요일이용비중", "order_type": "lowest"  }}, {"Usage_Count_Range": "상위 15%", "ranking_condition": {"filter_type": "None", "order_type": "None" }}, {"Average_Spending_Amount_Range": "상위 20%", "ranking_condition": {"filter_type": "None", "order_type": "None" }}, {"Spending_Amount_Range": "상위 80%", "ranking_condition": {"filter_type": "None", "order_type": "None" }}]

객체 개수 4
{'address': '제주시 노형동', 'category': '단품요리 전문점', 'ranking_condition': {'filter_type': '토요일이용비중', 'order_type': 'lowest'}}
{'Usage_Count_Range': '상위 15%', 'ranking_condition': {'filter_type': 'None', 'order_type': 'None'}}


In [7]:
# print(json_datas[2])

In [8]:
def merge_dicts(dicts):
    merged = {}

    for d in dicts:
        for key, value in d.items():
            if key == "ranking_condition" and key in merged:
                # ranking_condition의 filter_type이 None인 경우 넘어가고, 아니면 덮어씀
                if value["filter_type"] != "None":
                    merged[key].update(value)
            else:
                # 다른 경우는 그냥 덮어씀
                merged[key] = value

    return merged

In [12]:
json_data = json.loads(response.parts[0].text)
result = merge_dicts(json_data)
print(result)

{'address': '제주시 노형동', 'category': '단품요리 전문점', 'ranking_condition': {'filter_type': '토요일이용비중', 'order_type': 'lowest'}, 'Usage_Count_Range': '상위 15%', 'Average_Spending_Amount_Range': '상위 20%', 'Spending_Amount_Range': '상위 80%'}


In [10]:
# json_data = json.loads(response.parts[0].text)
# merge_dict = {}
# for data in json_data:
#     merge_dict.update(data)
# print(merge_dict)

In [11]:
try:
    result = merge_dicts(output)
    print(result)
except ValueError as e:
    print(e)

AttributeError: 'str' object has no attribute 'items'

In [10]:
output = response.parts[0].text
print(output)
print(response)

[{"Usage_Count_Range": "상위 10%", "Visit_count_specific": {"day": "토요일", "visit_percentage": 1.23}, "category": "단품요리 전문점", "ranking_condition": {"filter_type": "토요일이용비중", "order_type": "lowest" }}, {"Average_Spending_Amount_Range": "상위 20%", "address": "제주시 노형동", "category": "단품요리 전문점", "ranking_condition": {"filter_type": "토요일이용비중", "order_type": "lowest" }}]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "[{\"Usage_Count_Range\": \"\uc0c1\uc704 10%\", \"Visit_count_specific\": {\"day\": \"\ud1a0\uc694\uc77c\", \"visit_percentage\": 1.23}, \"category\": \"\ub2e8\ud488\uc694\ub9ac \uc804\ubb38\uc810\", \"ranking_condition\": {\"filter_type\": \"\ud1a0\uc694\uc77c\uc774\uc6a9\ube44\uc911\", \"order_type\": \"lowest\" }}, {\"Average_Spending_Amount_Range\": \"\uc0c1\uc704 20%\", \"address\": \"\uc81c\uc8fc